In [3]:
%cd /home/cthfvrl/Code/pairwise-similarities/
import pandas as pd 
kek = pd.read_csv('data/kaggle/code.csv', usecols=['doc'], lineterminator='\n')

/home/cthfvrl/Code/pairwise-similarities


In [30]:
%cd /home/cthfvrl/Code/pairwise-similarities/
import pandas as pd 
reader = pd.read_csv('data/kaggle/code.csv', chunksize=10**5, usecols=['doc'], lineterminator='\n')

/home/cthfvrl/Code/pairwise-similarities


In [26]:
size = 0
for chunk in reader:
    comments = list(map(text_to_word_sequence, comments))
    size += chunk.size
print(size)

3417339


In [49]:
from keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
df = reader.get_chunk(100)
comments = list(map(text_to_word_sequence, df['doc']))

In [50]:
word2vec = Word2Vec(comments, min_count=1, size=50)

In [55]:
X = "Asserts that two doubles are equal to within a positive delta.\
 If they are not, an AssertionError is thrown with the given\
 message. If the expected value is infinity then the delta value is\
 ignored. NaNs are considered equal:\
 assertEquals(Double.NaN, Double.NaN, *) passes\
 @param message the identifying message for the AssertionError (null\
 okay)\
 @param expected expected value\
 @param actual the value to check against expected\
 @param delta the maximum delta between expected and\
 actual for which both numbers are still\
 considered equal."
Y = "Asserts that two floats are equal to within a positive delta.\
 If they are not, an AssertionError is thrown. If the expected\
 value is infinity then the delta value is ignored. NaNs are considered\
 equal: assertEquals(Float.NaN, Float.NaN, *) passes\
 @param expected expected value\
 @param actual the value to check against expected\
 @param delta the maximum delta between expected and\
 actual for which both numbers are still\
 considered equal."
x = text_to_word_sequence(X)
y = text_to_word_sequence(Y)
-word2vec.wv.wmdistance(x, y)

-0.010170364950150551

In [58]:
from helpers import similarity_generator
wmd = similarity_generator.get_algorithm_by_name('WMD', True)
wmd.run_similarity([[X, Y]])

[-1.1024208237343307]

In [1]:
from metrics import metrics
from similarities.wmdk import WMDKSimilarity
metrics.train_test(metrics.get_metrics)('data/train.csv', 'data/test.csv', WMDKSimilarity(), True, 'WMDK')

Using TensorFlow backend.
Threshold = -5.9023, train accuracy = 0.9553


{'accuracy': 0.9347826086956522,
 'f1': 0.9312320916905444,
 'precision': 0.9848484848484849,
 'roc': 0.9959312972589792,
 'tp': 325,
 'tn': 363,
 'fp': 5,
 'fn': 43}

In [2]:
from metrics import x_feature_metrics

x_feature_metrics.lr_evaluate('data/train.csv', 'data/test.csv', WMDKSimilarity(), 'WMDK')

{'accuracy': 0.9646739130434783,
 'precision': 0.9803370786516854,
 'f1': 0.9640883977900553,
 'tp': 349,
 'tn': 361,
 'fp': 7,
 'fn': 19}

In [1]:
from metrics import metrics
from similarities.wmdk import WMDKSimilarity
metrics.train_test(metrics.get_metrics)('data/train.csv', 'data/test.csv', WMDKSimilarity(), True, 'WMDK')

Using TensorFlow backend.
Threshold = -8.7397, train accuracy = 0.9553


{'accuracy': 0.9320652173913043,
 'f1': 0.92816091954023,
 'precision': 0.9847560975609756,
 'roc': 0.9955030127599244,
 'tp': 323,
 'tn': 363,
 'fp': 5,
 'fn': 45}

In [2]:
from metrics import x_feature_metrics

x_feature_metrics.lr_evaluate('data/train.csv', 'data/test.csv', WMDKSimilarity(), 'WMDK')

{'accuracy': 0.9633152173913043,
 'precision': 0.9802816901408451,
 'f1': 0.9626556016597511,
 'tp': 348,
 'tn': 361,
 'fp': 7,
 'fn': 20}

In [1]:
from tabulate import tabulate

plain_d = {'accuracy': 0.9320652173913043,
 'f1': 0.92816091954023,
 'precision': 0.9847560975609756,
 'roc': 0.9955030127599244,
 'tp': 323,
 'tn': 363,
 'fp': 5,
 'fn': 45}
lr_d = {'accuracy': 0.9633152173913043,
 'precision': 0.9802816901408451,
 'f1': 0.9626556016597511,
 'tp': 348,
 'tn': 361,
 'fp': 7,
 'fn': 20}

table = []
headers = ['Algorithm', 'Accuracy', 'F1', 'Precision', 'TP', 'TN', 'FP', 'FN']
name = 'WMDK'
for d, n in (plain_d, name), (lr_d, name+' +'):
    table.append((n, d['accuracy'], d['f1'], d['precision'], d['tp'], d['tn'], d['fp'], d['fn']))
print(tabulate(table, headers, tablefmt='grid', floatfmt='.4f'))

+-------------+------------+--------+-------------+------+------+------+------+
| Algorithm   |   Accuracy |     F1 |   Precision |   TP |   TN |   FP |   FN |
+=============+============+========+=============+======+======+======+======+
| WMDK        |     0.9321 | 0.9282 |      0.9848 |  323 |  363 |    5 |   45 |
+-------------+------------+--------+-------------+------+------+------+------+
| WMDK +      |     0.9633 | 0.9627 |      0.9803 |  348 |  361 |    7 |   20 |
+-------------+------------+--------+-------------+------+------+------+------+


In [2]:
from helpers.similarity_generator import get_algorithm_by_name
from pipeline import ml

name = 'WMDK'
alg = get_algorithm_by_name(name, True)
model = ml.logistic_regression_train('data/train.csv', alg)
X, _ = ml.extract_features('data/unlabeled/junit4.csv', alg)

predictions = model.predict(X)
scores = model.predict_proba(X)
scores = scores[predictions == 1]
scores = scores[:, 1]

NameError: name 'df' is not defined

In [4]:
import pandas as pd
df = pd.read_csv('data/unlabeled/junit4.csv', index_col=0, na_filter=False)
df_up = df[predictions == 1]
df_up.drop(columns=['label', 'name1', 'name2'], inplace=True)
df_up.insert(4, 'score', scores)
df_up.sort_values(by='score', ascending=False, inplace=True)
df_up.to_csv('data/up/scores/' + name + '.csv')

print(name, len(df_up.index))

WMDK 98


In [5]:
import pickle
with open('data/up/accepted/all.pkl', 'rb') as f:
    all = pickle.load(f)
import pandas as pd
df = pd.read_csv('data/up/scores/WMDK.csv', index_col=0)
labels = [int(i in all) for i in df.index]
df.insert(5, 'label', labels)

names = ['LCS', 'COS', 'LEV', 'LSH', 'WMD', 'SiamX']
df_all = pd.concat([pd.read_csv('data/up/scores/'+name+'.csv', index_col=0) for name in names]).drop(columns=['score']).drop_duplicates()
ix_all = df_all.index.values
ix = df.index.values
import numpy as np
unique = np.setdiff1d(ix, ix_all)
df.loc[unique, 'label'] = -1

df.to_csv('data/up/WMDK_labeled.csv')
df.groupby('label').count()

,comment1,comment2,meta1,meta2,score
label,,,,,
0,5,5,5,5,5
1,93,93,93,93,93
